# 对抗生成网络----GAN
- 1. GANs实际是有两个网络, 一个生成网络 $G$ ，一个辨别网络 $D$, 两者相互竞争。生成网络伪造数据传入辨别网络。辨别网络同时也接收真实数据，并判断所输入数据是真实的还是伪造的。生成网络持续训练为了躲过辨别网络的鉴别，它试图输出看上去像真实数据的数据。而辨别网络持续训练来分辨谁真谁假！ 最终平衡的结果是：生成网络伪造的数据让辨别网络无法区分。
![GAN diagram](images/gan_diagram.png)
 2. 上图是GANs一般的网络结构，使用 MNIST图片作为真实数据，潜在样本是一个随机向量，生成网络据此开始伪造图片。随着生成网络学习深入，它会知道如何从随机向量映射到可辨识的图片，从而愚弄辨别网络。
 3. 辨别网络最终输出是一个sigmoid激活值,0代表伪造图片，1代表真实图片。 如果我们只关心生成新的图片，我们可以在训练完成后丢掉辨识网络即可。
 - 网络结构
 ![GAN diagram](images/gan_network.png)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
from IPython import display

- ### 加载数据集

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print("x_train data shape: ", x_train.shape)
print("y_train data shape: ", y_trian.shape)

- 数据预处理

In [ ]:
## 转换数据格式
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5
x_train = tf.expand_dims(x_train, -1)

- ### 构建生成器

In [ ]:
def model_generator():
    input_x = tf.keras.Input((100,))
    fc = tf.keras.layers.Dense(784, activation=None)(input_x)
    fc = tf.keras.layers.LeakyRelu()(fc)
    output = tf.keras.activations.tanh()(fc)
    model = tf.keras.Model(inputs=input_x, outputs=output)
    return model

- ### 构建判别器

In [ ]:
def model_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.Dense(128, activation=None))
    model.add(tf.keras.layers.LeakyRelu())
    model.add(tf.keras.layers.Dense(1, activation=None))
    # model.add(tf.keras.activations.sigmoid())
    return model

- ### loss

In [ ]:
def dis_losses(real_output, fake_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    d_real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    d_fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    d_loss = d_real_loss + d_fake_loss
    return d_loss

def gene_losses(fake_output):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    g_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return g_loss

- optimizer

In [ ]:
discriminator_optimizer = tf.keras.optimizers.SGD(0.01)
generator_optimizer = tf.keras.optimizers.SGD(0.01)

- load generator and discriminator

In [ ]:
generator = model_generator()
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
discriminator = model_discriminator()
result = discriminator(generated_image)
print (result)

- checkpoint

In [ ]:
checkpoint_dir = "./models/"
checkpoint_path = os.path.join(checkpoint_dir, "ckpt")
checkpoints = tf.train.Checkpoint(discriminator_optimizer=discriminator_optimizer,
                                 generator_optimizer=generator_optimizer,
                                 discriminator=discriminator,
                                 generator=generator)

- 训练

In [ ]:
def train_step(images, batch_size, noise_dim):
    noise = tf.random.normal([batch_size, noise_dim])
    with tf.GradientTape() as dis_tape, tf.GradientTape() as gen_tape:
        gene_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(gene_images, training=True)
        ## loss
        d_loss = dis_losses(real_output, fake_output)
        g_loss = gene_losses(fake_output)
    grad_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
    grad_discriminator = dis_tape.gradient(d_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(grad_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(grad_discriminator, discriminator.trainbale_variables))

In [ ]:
def train(train_data, epoches, batch_size, noise_dim):
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data).shuffle(256).batch(batch_size)
    for epoch in epoches:
        for batch in train_dataset:
            train_step(batch, batch_size, noise_dim)